In [2]:
import pandas as pd

In [65]:
inflation = pd.read_csv('../data/inflation_fred.csv')

In [70]:
inflation.head()

,inflation,date
0,29.01,1959-01-01
1,29.00,1959-02-01
2,28.97,1959-03-01
3,28.98,1959-04-01
4,29.04,1959-05-01


In [67]:
# changing to datetime format as the other dataset 
inflation['date'] = pd.to_datetime(inflation['observation_date'], format='%m/%d/%Y', errors='coerce')

In [68]:
inflation = inflation.drop(columns=['observation_date'])

In [69]:
inflation = inflation.rename(columns={'CPIAUCSL': 'inflation'})

In [7]:
unemployment = pd.read_csv('../data/unemployment_bls.csv', skiprows = 11)

In [8]:
unemployment.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1959,6.0,5.9,5.6,5.2,5.1,5.0,5.1,5.2,5.5,5.7,5.8,5.3
1,1960,5.2,4.8,5.4,5.2,5.1,5.4,5.5,5.6,5.5,6.1,6.1,6.6
2,1961,6.6,6.9,6.9,7.0,7.1,6.9,7.0,6.6,6.7,6.5,6.1,6.0
3,1962,5.8,5.5,5.6,5.6,5.5,5.5,5.4,5.7,5.6,5.4,5.7,5.5
4,1963,5.7,5.9,5.7,5.7,5.9,5.6,5.6,5.4,5.5,5.5,5.7,5.5


In [9]:
month_mapping = {'Jan': '01', 'Feb': '02', 'Mar': '03',
                 'Apr': '04', 'May': '05', 'Jun': '06',
                 'Jul': '07', 'Aug': '08', 'Sep': '09',
                 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [10]:
unemployment_long = pd.melt(unemployment, 
        id_vars = ['Year'],
        var_name = 'Month', 
        value_name= 'unemployment_rate'
        )

In [11]:
unemployment_long['month_num'] = unemployment_long['Month'].map(month_mapping)

In [12]:
unemployment_long['date'] =unemployment_long['Year'].astype(str) + '-' + unemployment_long['month_num'] 

In [13]:
unemployment_long['date'] = pd.to_datetime(unemployment_long['date'], format='%Y-%m', errors='coerce')

In [14]:
unemployment_long = unemployment_long.drop(columns = ['Year', 'Month', 'month_num'])

In [15]:
unemployment_long.head()

,unemployment_rate,date
0,6.0,1959-01-01
1,5.2,1960-01-01
2,6.6,1961-01-01
3,5.8,1962-01-01
4,5.7,1963-01-01


In [16]:
debt = pd.read_csv('../data/debt_fed.csv')

In [17]:
debt.head(15)

,Series Description,"Percent change of total consumer credit, seasonally adjusted at an annual rate","Percent change of total revolving consumer credit, seasonally adjusted at an annual rate","Percent change of total nonrevolving consumer credit, seasonally adjusted at an annual rate","Total consumer credit owned and securitized, seasonally adjusted level","Revolving consumer credit owned and securitized, seasonally adjusted level","Nonrevolving consumer credit owned and securitized, seasonally adjusted level","Total consumer credit owned and securitized, not seasonally adjusted level","Total securitized consumer credit, not seasonally adjusted level","Total consumer credit owned and securitized by depository institutions, not seasonally adjusted level",...,"Finance rate on personal loans at commercial banks, 24 month loan; not seasonally adjusted","Commercial bank interest rate on credit card plans, all accounts; not seasonally adjusted","Commercial bank interest rate on credit card plans, accounts assessed interest; not seasonally adjusted","New car average finance rate at auto finance companies, not seasonally adjusted (discontinued after January 2011)","Weighted-average maturity of new car loans at auto finance companies, not seasonally adjusted (discontinued after January 2011)","Loan-to-value ratio of new car loans at auto finance companies, not seasonally adjusted (discontinued after January 2011)","Average amount financed for new car loans at auto finance companies, not seasonally adjusted (discontinued after January 2011)","Average maturity of new car loans at finance companies, amount of finance weighted, not seasonally adjusted","Average finance rate of new car loans at finance companies, amount of finance weighted, not seasonally adjusted","Average amount financed for new car loans at finance companies, not seasonally adjusted"
0,Unit:,Percent,Percent,Percent,Currency,Currency,Currency,Currency,Currency,Currency,...,Percent,Percent,Percent,Percent,Months,Percent,Currency,Months,Percent,Currency
1,Multiplier:,1,1,1,1000000,1000000,1000000,1000000,1000000,1000000,...,1,1,1,1,1,1,1,1,1,1
2,Currency:,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
3,Unique Identifier:,G19/CCOUT/DTCTL_@%A_BA.M,G19/CCOUT/DTCTLR_@%A_BA.M,G19/CCOUT/DTCTLN_@%A_BA.M,G19/CCOUT/DTCTL.M,G19/CCOUT/DTCTLR.M,G19/CCOUT/DTCTLN.M,G19/CCOUT/DTCTL_N.M,G19/CCOUT/DTCNL_N.M,G19/CCOUT/DTCTLHD_N.M,...,G19/TERMS/RIFLPBCIPLM24_N.M,G19/TERMS/RIFSPBCICC_N.M,G19/TERMS/RIFSPBCICCI_N.M,G19/TERMS/H0.RIFLPCFAN_N.M,G19/TERMS/H0.DTCTLVNM_N.M,G19/TERMS/H0.DTCTLVNL_N.M,G19/TERMS/H0.DTCTLVNA_N.M,G19/TERMS/DTCTLVENM_N.M,G19/TERMS/RIELPCFAN_N.M,G19/TERMS/DTCTLVENA_N.M
4,Time Period,DTCTL_@%A_BA.M,DTCTLR_@%A_BA.M,DTCTLN_@%A_BA.M,DTCTL.M,DTCTLR.M,DTCTLN.M,DTCTL_N.M,DTCNL_N.M,DTCTLHD_N.M,...,RIFLPBCIPLM24_N.M,RIFSPBCICC_N.M,RIFSPBCICCI_N.M,H0.RIFLPCFAN_N.M,H0.DTCTLVNM_N.M,H0.DTCTLVNL_N.M,H0.DTCTLVNA_N.M,DTCTLVENM_N.M,RIELPCFAN_N.M,DTCTLVENA_N.M
5,1943-01,NaN,NaN,NaN,6577.83,NaN,6577.83,6550,NaN,2554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1943-02,-20.94,NaN,-20.94,6463.04,NaN,6463.04,6276,NaN,2444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1943-03,-42.49,NaN,-42.49,6234.21,NaN,6234.21,6099,NaN,2376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1943-04,-20.88,NaN,-20.88,6125.75,NaN,6125.75,6065,NaN,2324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1943-05,-37.12,NaN,-37.12,5936.26,NaN,5936.26,5967,NaN,2278,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
debt.columns = debt.columns.str.strip()

debt = debt[['Series Description', 
             'Revolving consumer credit owned and securitized, seasonally adjusted level', 
             'Nonrevolving consumer credit owned and securitized, seasonally adjusted level', 
             'Total consumer credit owned and securitized, seasonally adjusted level']]

In [19]:
# removing the first five rows to remove informational rows
debt = debt.iloc[5:]

In [20]:
debt['date'] = pd.to_datetime(debt['Series Description'], format='%Y-%m', errors='coerce')

In [21]:
debt = debt.drop(columns=['Series Description'])

In [22]:
debt.rename(columns = {
    'Revolving consumer credit owned and securitized, seasonally adjusted level': 'revolving_credit',
    'Nonrevolving consumer credit owned and securitized, seasonally adjusted level': 'nonrevolving_credit',
    'Total consumer credit owned and securitized, seasonally adjusted level': 'total_credit'
}, inplace=True)


In [23]:
per_cons = pd.read_csv('../data/pce_bea.csv', skiprows = 3, header = None)

In [24]:
years = per_cons.iloc[0]
months = per_cons.iloc[1]
combined_date = months + ' ' + years

In [25]:
date = pd.to_datetime(combined_date, format = '%b %Y', errors='coerce')

In [26]:
per_cons.columns = date
per_cons.columns = per_cons.columns.astype(str)

In [27]:
per_cons.columns = per_cons.columns.astype(str)
per_cons.columns.values[1] = 'description'

In [28]:
per_cons['description'] = per_cons['description'].str.strip()

In [29]:
per_cons = per_cons[per_cons['description'] == 'Personal consumption expenditures']

In [30]:
per_cons.head(35)

,NaT,description,1959-01-01,1959-02-01,1959-03-01,1959-04-01,1959-05-01,1959-06-01,1959-07-01,1959-08-01,...,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01
30,29,Personal consumption expenditures,306.1,309.6,312.7,312.2,316.1,318.2,317.8,320.2,...,19866.3,19905.0,20044.1,20123.2,20235.1,20408.1,20389.0,20469.3,20621.8,20669.5


In [31]:
per_con_long = pd.melt(
    per_cons, 
    id_vars = None,
    var_name = 'date',
    value_name = 'pce'
)

In [32]:
per_con_long = per_con_long.iloc[2:]

In [53]:
per_con_long['date'] = pd.to_datetime(per_con_long['date'])

In [54]:
per_con_long

,date,pce
2,1959-01-01,306.1
3,1959-02-01,309.6
4,1959-03-01,312.7
5,1959-04-01,312.2
6,1959-05-01,316.1
...,...,...
793,2024-12-01,20408.1
794,2025-01-01,20389.0
795,2025-02-01,20469.3
796,2025-03-01,20621.8


In [71]:
# Check data types for each DataFrame
print("Data types in per_con_long:")
print(per_con_long.dtypes)

print("\nData types in inflation:")
print(inflation.dtypes)

print("\nData types in unemployment_long:")
print(unemployment_long.dtypes)

print("\nData types in debt:")
print(debt.dtypes)

Data types in per_con_long:
date    datetime64[ns]
pce             object
dtype: object

Data types in inflation:
inflation           float64
date         datetime64[ns]
dtype: object

Data types in unemployment_long:
unemployment_rate           float64
date                 datetime64[ns]
dtype: object

Data types in debt:
revolving_credit               object
nonrevolving_credit            object
total_credit                   object
date                   datetime64[ns]
dtype: object


In [72]:
combined_df = per_con_long.merge(
    inflation, 
    how='left', 
    on='date'
).merge(
    unemployment_long, 
    how='left', 
    on='date'
).merge(
    debt, 
    how='left', 
    on='date'
)

In [73]:
combined_df

,date,pce,inflation,unemployment_rate,revolving_credit,nonrevolving_credit,total_credit
0,1959-01-01,306.1,29.010,6.0,NaN,48961.16,48961.16
1,1959-02-01,309.6,29.000,5.9,NaN,49513.71,49513.71
2,1959-03-01,312.7,28.970,5.6,NaN,50007.73,50007.73
3,1959-04-01,312.2,28.980,5.2,NaN,50463.43,50463.43
4,1959-05-01,316.1,29.040,5.1,NaN,51007.24,51007.24
...,...,...,...,...,...,...,...
791,2024-12-01,20408.1,317.603,4.1,1316865.89,3671015.14,4987881.03
792,2025-01-01,20389.0,319.086,4.0,1323514.25,3673358.72,4996872.97
793,2025-02-01,20469.3,319.775,4.1,1323945.56,3672057.95,4996003.5
794,2025-03-01,20621.8,319.615,4.2,1313730.92,3678871.89,4992602.81


In [74]:
combined_df.to_csv('../data/dataformodel.csv', index=False)